In [ ]:
import json,os,time
import torch
import numpy as np
name_list=['hog','hof','mbhx','mbhy','tra']
feature_list=[96,108,96,96,30]
feature_cumsum_list=[0]
for i in feature_list:
    feature_cumsum_list.append(feature_cumsum_list[-1]+i)
data_path='/kaggle/input/hmdb51'
abspath=os.path.abspath('')
print(os.listdir(data_path))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
central_dict=np.load(os.path.join(data_path,'centers_all.npz'))
central=torch.from_numpy(np.expand_dims(np.concatenate([central_dict[i] for i in name_list],axis=1),0)).to(device)
print(central.shape)
def bag_of_feature(filename):
    X=torch.from_numpy(np.expand_dims(np.load(filename).astype(np.float32),1)).to(device)
    shape0=X.shape[0]
#     print(shape0)
    max_num=256
    iteration_num=(shape0-1)//max_num+1
    feature=torch.zeros((len(name_list),4000),dtype=torch.int32,device=device)
    for i in range(iteration_num):
        sub=X[i*max_num:min((i+1)*max_num,shape0)]-central
        sub.square_()
        for i in range(len(name_list)):
            torch_sum=torch.sum(sub[:,:,feature_cumsum_list[i]:feature_cumsum_list[i+1]],dim=2)
            arg=torch.argmin(torch_sum,dim=1)
            feature[i].add_(torch.bincount(arg,minlength=4000))
#     print(torch.sum(feature))
    if device!='cpu':
        return feature.cpu()
    return feature
        #arg=torch.argmin(torch.sum(torch.square(),axis=2),axis=1)
        #feature+=np.bincount(arg,minlength=4000+class_num)
    #feature=torch.zeros((len(name_list),4000),dtype=torch.int32,device=device)
    #for 
    #for i in range(len(name_list)):
        
    
def deal(path_list):
    #central=np.load()
    for i in path_list:
        path1=os.path.join(data_path,i)
        if len(os.listdir(path1))==1:
            path1=os.path.join(path1,i)
        feature_dict_file=os.path.join(abspath,'feature_%s.npz' %i)
        print(feature_dict_file)
        print(os.path.exists(feature_dict_file))
        time2=time.time()
        if os.path.exists(feature_dict_file):
            feature_dict=dict(np.load(feature_dict_file))
            print(len(feature_dict))
        else:
            feature_dict={}
        print(len(os.listdir(path1)))
        for j in os.listdir(path1):
            #time0=time.time()
            if j not in feature_dict:
                path2=os.path.join(path1,j)
                feature_dict[j]=bag_of_feature(path2)
                np.savez(feature_dict_file,**feature_dict)
        time3=time.time()
            #time1=time.time()
            #print(time1-time0)
        print(time3-time2)
a=os.listdir(data_path)
d= ['stand', 'smoke', 'somersault', 'talk', 'sword', 'sword_exercise', 'walk', 'shoot_gun', 'smile', 'situp', 'sit', 'swing_baseball', 'throw', 'wave', 'turn']
b=[i  for i in os.listdir(data_path) if ((i not in d) and ('.' not in i) )]
print(len(b))
deal(b)

In [ ]:
import zipfile,os

#abspath='/content/drive/MyDrive/Colab Notebooks/action_recognition/hmdb51'
# zippath=os.path.join(abspath,'video.zip')
# videopath=os.path.join(abspath,'videofile')
# #创建zip压缩包
# zip_file = zipfile.ZipFile(
#     zippath, 'w'
# )
# #将key文件写入zip压缩包中
l=os.listdir(abspath)
zippath=os.path.join(abspath,'npz.zip')
zip_file = zipfile.ZipFile(
    zippath, 'w'
)
for i in l:
  zip_file.write(os.path.join(abspath,i),i)
zip_file.close()